In [1]:
import torch
from torch import nn
from utils import *

from model_cnn import SiameseCNN
import dlc_practical_prologue as prologue

import pdb

In [2]:
train_input, train_target, train_classes,test_input, test_target, test_classes= prologue.generate_pair_sets(1000)

In [3]:
train_classes_oh, train_target_oh = to_one_hot(train_classes),to_one_hot(train_target) # size of 1000*2*10
test_classes_oh, test_target_oh = to_one_hot(test_classes),to_one_hot(test_target)

In [4]:
train_dataset =  DatasetLoader(train_input, train_target_oh, train_classes_oh,batch_size = 100,shuffle=True)

In [5]:
def run(inputdata,input_target,input_classes, train = True, test = False, model=SiameseCNN(), aux_loss=True):
    if train :
        learning_rate = 1e-3
        batch_size =100
        n_epoch = 100
        model.apply(initialize_weights)
        loss_fn =  nn.BCELoss()
        
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
        StepLR = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.65)
        
        losses = []
        for i in range(n_epoch):
            total_loss = 0

            for index, (batch_data,y,batch_classes) in enumerate(train_dataset ):
                if aux_loss :

                    classes, pred = model(batch_data)

                    label0 = batch_classes[0]
                    label1 = batch_classes[1]

                    loss =loss_fn(classes[0],label0)+loss_fn(classes[1],label1)+loss_fn(pred,y)

                if not(aux_loss ):
                    pred = model(batch_data,aux_loss)
                    loss =  loss_fn(pred,y)



                # Back propagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                with torch.no_grad():
                    total_loss = total_loss+loss

            losses.append(total_loss/float(index+1))
            classes, pred = model(inputdata)
#             _,pred_classes = torch.max(classes[0],1)
#             _,true_classes = torch.max(input_classes[:,0,:],1)
            pred_target = torch.zeros(1000,1)

            _,pred_target = torch.max(pred,1)
            _,true_target = torch.max(input_target,1)
            
            if i%5 == 0:
                print('--The average loss of',i,' epoch is ',losses[i])
                print('--The batch accurancy is',torch.sum(pred_target.view(-1)==true_target.type(torch.float))/1000.0)
        torch.save(model, './model_parameters/siamese.pkl') 
        return losses
    
    if test:
        model = torch.load('./model_parameters/siamese.pkl')
        classes, pred = model(inputdata)
        _,pred_classes = torch.max(classes[0],1)
        _,true_classes = torch.max(input_classes[:,0,:],1)
        pred_target = torch.zeros(1000,1)
        
        _,pred_target = torch.max(pred,1)
        _,input_target = torch.max(input_target,1)

        print('accurancy of test process-----',torch.sum(pred_target.view(-1)==input_target.type(torch.float))/1000.0)
        return pred_target

In [6]:
# training network
losses = run(train_input, train_target_oh, train_classes_oh, train = True, test = False, model=SiameseCNN(), aux_loss=False)

--The average loss of 0  epoch is  tensor(0.6842)
--The batch accurancy is tensor(0.6550)
--The average loss of 5  epoch is  tensor(0.6584)
--The batch accurancy is tensor(0.7390)
--The average loss of 10  epoch is  tensor(0.5723)
--The batch accurancy is tensor(0.7470)
--The average loss of 15  epoch is  tensor(0.5474)
--The batch accurancy is tensor(0.7830)
--The average loss of 20  epoch is  tensor(0.5124)
--The batch accurancy is tensor(0.8060)
--The average loss of 25  epoch is  tensor(0.4555)
--The batch accurancy is tensor(0.8220)
--The average loss of 30  epoch is  tensor(0.4651)
--The batch accurancy is tensor(0.8290)
--The average loss of 35  epoch is  tensor(0.4198)
--The batch accurancy is tensor(0.8510)
--The average loss of 40  epoch is  tensor(0.3611)
--The batch accurancy is tensor(0.8710)
--The average loss of 45  epoch is  tensor(0.3147)
--The batch accurancy is tensor(0.8740)
--The average loss of 50  epoch is  tensor(0.2860)
--The batch accurancy is tensor(0.8540)
-

In [7]:

result = run(test_input, test_target_oh, test_classes_oh, train = False, test = True, model=SiameseCNN(), aux_loss=True)

accurancy of test process----- tensor(0.7770)
